
## Introduction
Le Jeu de la Vie, conceptualisé par John Conway en 1970, offre une perspective fascinante sur l'émergence de la complexité à partir de règles simples dans un environnement cellulaire. Chaque cellule évolue en fonction d'un ensemble de règles, donnant naissance à des motifs dynamiques et souvent imprévisibles. Cette implémentation en Python avec Tkinter explore différentes variantes du jeu, chacune ayant ses propres règles, offrant ainsi une opportunité d'observer comment ces variations influent sur les évolutions et les structures émergentes.

## Enjeux de la Comparaison des Règles
Compréhension des Dynamiques Émergentes
Chaque variante du Jeu de la Vie a ses règles spécifiques, impactant directement les dynamiques du système. Comparer ces règles permet de mieux comprendre comment des variations simples peuvent conduire à des comportements émergents complexes. Cela nous aide à explorer la diversité des phénomènes qui peuvent émerger à partir de règles sous-jacentes distinctes.

# Exploration des Configurations Initiales
En observant comment différentes configurations initiales réagissent sous différentes règles, nous pouvons étudier comment les règles influent sur la formation de structures stables, d'oscillations ou de motifs mobiles. Chaque règle peut favoriser ou inhiber certains types de configurations, offrant ainsi une perspective approfondie sur la diversité des résultats possibles.

# Comparaison des Stratégies de Survie et de Naissance
Chaque variante a ses propres stratégies de survie et de naissance pour les cellules. Comparer ces stratégies permet d'explorer comment des ajustements subtils dans les règles peuvent conduire à des résultats très différents. Certains jeux peuvent favoriser la stabilité, tandis que d'autres encouragent des évolutions plus dynamiques.

# Impact sur la Recherche Scientifique
La comparaison des différentes règles du Jeu de la Vie n'est pas seulement un exercice ludique, mais elle peut également avoir des implications dans des domaines tels que la biologie théorique, la modélisation de systèmes complexes, et la compréhension des principes sous-jacents aux phénomènes naturels.

En explorant ces variations, nous pouvons enrichir notre compréhension des systèmes dynamiques, de la complexité émergente et de la manière dont de simples règles peuvent conduire à des résultats diversifiés.

In [ ]:
import tkinter as tk

class GameOfLife:
    def __init__(self, master, cell_size, width=800, height=600, speed=50):
        self.master = master
        self.master.title("Jeu de la Vie")

        self.cell_size = cell_size
        self.width = width
        self.height = height
        self.speed = speed
        self.is_running = False
        self.is_setup = False
        self.current_rule = "Conway's Game of Life"

        # Initialize the rules dictionary
        self.rules = {
            "Conway's Game of Life": self.apply_conways_rule,
            "Day and Night": self.apply_day_and_night_rule,
            "HighLife": self.apply_highlife_rule,
            "Immigration": self.apply_immigration_rule,
            "QuadLife": self.apply_quadlife_rule
        }

        self.create_widgets()
        self.create_grid()

    def create_widgets(self):
        self.canvas = tk.Canvas(self.master, width=self.width, height=self.height, bg='white')
        self.canvas.bind("<Button-1>", self.toggle_cell)
        self.canvas.pack()

        self.start_button = tk.Button(self.master, text='Go!', command=self.toggle_simulation)
        self.start_button.pack(side=tk.LEFT, padx=5)
        
        self.stop_button = tk.Button(self.master, text='Stop', command=self.stop_simulation)
        self.stop_button.pack(side=tk.LEFT, padx=5)

        self.reset_button = tk.Button(self.master, text='Reset Grid', command=self.reset_grid)
        self.reset_button.pack(side=tk.LEFT, padx=5)

        self.speed_label = tk.Label(self.master, text='Speed (ms):')
        self.speed_label.pack(side=tk.RIGHT)
        
        self.speed_entry = tk.Entry(self.master)
        self.speed_entry.insert(tk.END, str(self.speed))
        self.speed_entry.bind("<Return>", self.change_speed)
        self.speed_entry.pack(side=tk.RIGHT)

        self.status_label = tk.Label(self.master, text='Setup')
        self.status_label.pack(side=tk.BOTTOM)

        self.rule_label = tk.Label(self.master, text='Rule: Conway\'s Game of Life')
        self.rule_label.pack(side=tk.TOP)
        self.rule_buttons = []
        rules = ["Conway's Game of Life", "Day and Night", "HighLife", "Immigration", "QuadLife"]
        for rule in rules:
            rule_button = tk.Button(self.master, text=rule, command=lambda r=rule: self.set_rule(r))
            rule_button.pack(side=tk.TOP, padx=5)
            self.rule_buttons.append(rule_button)

        self.set_rule("Conway's Game of Life")

    def create_grid(self):
        self.cells = {}
        for x in range(0, self.width, self.cell_size):
            for y in range(0, self.height, self.cell_size):
                cell_id = self.canvas.create_rectangle(x, y, x + self.cell_size, y + self.cell_size, fill='white')
                self.cells[x, y] = cell_id

    def toggle_cell(self, event):
        x, y = event.x - (event.x % self.cell_size), event.y - (event.y % self.cell_size)
        cell_id = self.cells[x, y]
        fill_color = 'black' if self.canvas.itemcget(cell_id, 'fill') == 'white' else 'white'
        self.canvas.itemconfig(cell_id, fill=fill_color)

    def toggle_simulation(self):
        if self.is_setup:
            self.status_label.config(text='Playing')
            self.is_running = True
            self.run_simulation()
        else:
            self.status_label.config(text='Setup')
            self.is_setup = True

    def run_simulation(self):
        self.play()
        if self.is_running:
            self.master.after(self.speed, self.run_simulation)

    def stop_simulation(self):
        self.is_running = False
        self.status_label.config(text='Setup')

    def play(self):
        new_cells = {}
        for x in range(0, self.width, self.cell_size):
            for y in range(0, self.height, self.cell_size):
                neighbors = self.count_neighbors(x, y)
                cell_id = self.cells[x, y]

                new_cells[x, y] = self.rules[self.current_rule](cell_id, neighbors)

        for x, y in new_cells:
            cell_id = self.cells[x, y]
            self.canvas.itemconfig(cell_id, fill=new_cells[x, y])

    def count_neighbors(self, x, y):
        neighbors = 0
        for i in [-self.cell_size, 0, self.cell_size]:
            for j in [-self.cell_size, 0, self.cell_size]:
                if i == j == 0:
                    continue
                neighbor_x, neighbor_y = (x + i) % self.width, (y + j) % self.height
                cell_id = self.cells[neighbor_x, neighbor_y]
                if self.canvas.itemcget(cell_id, 'fill') == 'black':
                    neighbors += 1
        return neighbors

    def reset_grid(self):
        self.is_setup = False
        self.status_label.config(text='Setup')
        for x in range(0, self.width, self.cell_size):
            for y in range(0, self.height, self.cell_size):
                cell_id = self.cells[x, y]
                self.canvas.itemconfig(cell_id, fill='white')

    def change_speed(self, event):
        try:
            self.speed = int(self.speed_entry.get())
        except ValueError:
            pass

    def set_rule(self, rule):
        self.current_rule = rule
        self.rule_label.config(text=f'Rule: {rule}')

    def apply_conways_rule(self, cell_id, neighbors):
        if self.canvas.itemcget(cell_id, 'fill') == 'white' and neighbors == 3:
            return 'black'
        elif self.canvas.itemcget(cell_id, 'fill') == 'black' and (neighbors < 2 or neighbors > 3):
            return 'white'
        else:
            return self.canvas.itemcget(cell_id, 'fill')

    def apply_day_and_night_rule(self, cell_id, neighbors):
        if self.canvas.itemcget(cell_id, 'fill') == 'white' and (neighbors == 3 or neighbors == 6 or neighbors == 7 or neighbors == 8):
            return 'black'
        elif self.canvas.itemcget(cell_id, 'fill') == 'black' and (neighbors != 3 and neighbors != 6 and neighbors != 7 and neighbors != 8):
            return 'white'
        else:
            return self.canvas.itemcget(cell_id, 'fill')

    def apply_highlife_rule(self, cell_id, neighbors):
        if self.canvas.itemcget(cell_id, 'fill') == 'white' and (neighbors == 3 or neighbors == 6):
            return 'black'
        elif self.canvas.itemcget(cell_id, 'fill') == 'black' and (neighbors != 2 and neighbors != 3):
            return 'white'
        else:
            return self.canvas.itemcget(cell_id, 'fill')

    def apply_immigration_rule(self, cell_id, neighbors):
        alive_neighbors = neighbors
        if self.canvas.itemcget(cell_id, 'fill') == 'black':
            alive_neighbors -= 1
        if alive_neighbors in {3, 5, 6, 7, 8}:
            return 'black'
        else:
            return 'white'

    def apply_quadlife_rule(self, cell_id, neighbors):
        if self.canvas.itemcget(cell_id, 'fill') == 'white' and (neighbors == 2 or neighbors == 3):
            return 'black'
        elif self.canvas.itemcget(cell_id, 'fill') == 'black' and (neighbors < 2 or neighbors > 3):
            return 'white'
        else:
            return self.canvas.itemcget(cell_id, 'fill')

if __name__ == "__main__":
    root = tk.Tk()
    game = GameOfLife(root, cell_size=20)
    root.mainloop()


## Comparaison
# Conway's Game of Life: 
Il tend à générer des oscillations, des motifs stables, et des vaisseaux spatiaux.

# Day and Night: 
Cette variante peut produire des motifs oscillants intéressants, parfois avec des périodes différentes par rapport à Conway's Game of Life.

# HighLife: 
Elle a des règles qui favorisent la création de structures stables et de motifs oscillants.

# Immigration: 
Elle a des règles uniques qui peuvent générer des structures stables et des oscillations.

# QuadLife: 
Elle a des caractéristiques similaires à Conway's Game of Life mais avec des résultats différents.